### Converting between instruments using a Seq2Seq Model

In [1]:
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import librosa
import numpy as np
import random

from pathlib import Path
from torch.utils.data import Dataset

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Torchaudio processing methods

In [3]:
def normalized(tensor):
    # https://pytorch.org/tutorials/beginner/audio_preprocessing_tutorial.html
    centered = tensor - tensor.mean()
    normalized = tensor / tensor.abs().max()
    return normalized

_mu_encoder = torchaudio.transforms.MuLawEncoding()
_mu_decoder = torchaudio.transforms.MuLawDecoding()

def mu_law_encode(waveform):
    return _mu_encoder(normalized(waveform))

def mu_law_decode(waveform):
    return _mu_decoder(normalized(waveform))

def load_audio(path):
    """ Load .wav file to Mu law encoding tensor """
    waveform, sample_rate = torchaudio.load(path)
    return mu_law_encode(waveform), sample_rate

def save_audio(path, data, sample_rate):
    """ Save Mu law encoding tensor to .wav file """
    waveform = mu_law_decode(data)
    torchaudio.save(path, data, sample_rate)

Converting wav files to tensors and saving them

In [33]:
dataset = Path.cwd().joinpath("dataset")
train_wav = dataset.joinpath("train_wav")
train_tensors = dataset.joinpath("train_tensors")
test_wav = dataset.joinpath("test_wav")
test_tensors = dataset.joinpath("test_tensors")

for instrument in train_wav.iterdir():
    if (instrument.name != ".DS_Store"):
        for note in train_wav.joinpath(instrument.name).iterdir():
            if (note.name != ".DS_Store"):
                wav_tensor = load_audio(train_wav.joinpath(instrument.name, note.name))
                wav_tensor = (wav_tensor[0][0] + wav_tensor[0][1]) / 2
                torch.save(wav_tensor, train_tensors.joinpath(instrument.name, note.stem + ".pt"))
                
for instrument in test_wav.iterdir():
    if (instrument.name != ".DS_Store"):
        for note in test_wav.joinpath(instrument.name).iterdir():
            if (note.name != ".DS_Store"):
                wav_tensor = load_audio(test_wav.joinpath(instrument.name, note.name))
                wav_tensor = (wav_tensor[0][0] + wav_tensor[0][1]) / 2
                torch.save(wav_tensor, test_tensors.joinpath(instrument.name, note.stem + ".pt"))

Loading in the tensors and preparing them for the dataset

In [34]:
dataset = Path.cwd().joinpath("dataset")

#Change for model you are training
train_input = dataset.joinpath("train_tensors", "75")
train_output = dataset.joinpath("train_tensors", "81")
test_input = dataset.joinpath("test_tensors", "75")
test_output = dataset.joinpath("test_tensors", "81")

train_input_wav = []
train_output_wav = []
test_input_wav = []
test_output_wav = []

#Take out DS_Stores if you are using a Mac
for tensor in train_input.iterdir():
    if tensor.name != ".DS_Store":
        train_input_wav.append(tensor)
    
for tensor in train_output.iterdir():
    if tensor.name != ".DS_Store":
        train_output_wav.append(tensor)

for tensor in test_input.iterdir():
    if tensor.name != ".DS_Store":
        test_input_wav.append(tensor)
    
for tensor in test_output.iterdir():
    if tensor.name != ".DS_Store":
        test_output_wav.append(tensor)
        
print("train lengths:", len(train_input_wav), len(train_output_wav))
print("test lengths:", len(test_input_wav), len(test_output_wav))

train lengths: 45 45
test lengths: 15 15


In [42]:
class Seq2SeqDataset(Dataset):
    """
    Seq2Seq Dataset. Uses torchaudio + mu law to process wav files.
    """

    def __init__(self, input_wavs, output_wavs, transform=None):
        """
        Args:
            input_wavs: list of input wavs from 1st instrument
            output_wavs: list of output wavs from 2nd instrument
        """
        self.input_wavs = input_wavs
        self.output_wavs = output_wavs
        
    def __len__(self):
        return len(self.input_wavs)

    def __getitem__(self, index):
        input = torch.load(self.input_wavs[index])
        output = torch.load(self.output_wavs[index])
        return input, output

Create data loaders using Seq2SeqDataset

In [43]:
train_set = Seq2SeqDataset(train_input_wav, train_output_wav)
test_set = Seq2SeqDataset(test_input_wav, test_output_wav)
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 8, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 8, shuffle = True, **kwargs)

Train set size: 45
Test set size: 15


In [44]:
for data, target in train_loader:
    print(data, target)

torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
tensor([[122, 122, 125,  ...,  16,  17,  19],
        [121, 121, 124,  ..., 103, 124, 142],
        [122, 122, 125,  ..., 204, 186,  70],
        ...,
        [122, 122, 125,  ...,  10,  11,  14],
        [121, 121, 124,  ...,   7,   7,   7],
        [122, 122, 125,  ...,  70,  86, 134]]) tensor([[124, 124, 126,  ..., 219, 219, 219],
        [124, 124, 126,  ...,  67,  63,  58],
        [124, 124, 126,  ..., 146, 145, 143],
        ...,
        [124, 124, 126,  ..., 143, 223, 232],
        [124, 124, 126,  ..., 101,  84,  37],
        [123, 123, 125,  ..., 213, 212, 211]])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
torch.Size([22208])
